In [1]:
!pip install evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 7.8 MB/s eta 0:00:00


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
import pandas as pd
import torch
from sklearn.model_selection import train_test_split
from datasets import Dataset
from transformers import (
    AutoTokenizer,
    DistilBertForSequenceClassification,
    Trainer,
    TrainingArguments,
    DataCollatorWithPadding
)
import evaluate
import numpy as np

df = pd.read_csv('/content/drive/MyDrive/data.csv').sample(frac=0.05, random_state=42).reset_index(drop=True)

# Rename label column and map to 0/1
df = df.rename(columns={"source": "label"})
df["label"] = df["label"].map(lambda x: 1 if x == "Human" else 0)

# Split train/test
train_df, test_df = train_test_split(df, test_size=0.1, random_state=42)

# Convert to Hugging Face Dataset
train_dataset = Dataset.from_pandas(train_df)
test_dataset = Dataset.from_pandas(test_df)

# Tokenization
model_name = "distilbert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_name)

def tokenize(batch):
    return tokenizer(
        batch["text"],
        padding="max_length",
        truncation=True,
        max_length=256
    )

train_dataset = train_dataset.map(tokenize, batched=True)
test_dataset = test_dataset.map(tokenize, batched=True)

# Rename labels column
train_dataset = train_dataset.rename_column("label", "labels")
test_dataset = test_dataset.rename_column("label", "labels")

# Remove unnecessary columns
remove_cols = [c for c in ['prompt_id', 'text_length', 'word_count', '__index_level_0__', 'text']
               if c in train_dataset.column_names]
train_dataset = train_dataset.remove_columns(remove_cols)
test_dataset = test_dataset.remove_columns(remove_cols)

# Model

model = DistilBertForSequenceClassification.from_pretrained(model_name, num_labels=2)

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

# Place model on GPU if available
device = "cuda" if torch.cuda.is_available() else "cpu"

metric = evaluate.load("f1")

def compute_metrics(eval_pred):
    print("compute_metrics called!", flush=True)
    logits, labels = eval_pred
    preds = np.argmax(logits, axis=1)
    result = metric.compute(predictions=preds, references=labels)
    print(f"Result from metric.compute: {result}", flush=True)
    return result

args = TrainingArguments(
    output_dir="./results",
    eval_strategy="epoch",
    save_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=3,
    weight_decay=0.01,
    load_best_model_at_end=True,
    metric_for_best_model="f1",
    greater_is_better=True,
    gradient_accumulation_steps=1,
    fp16=False,
    dataloader_num_workers=0,
    remove_unused_columns=False,
    report_to="none"
)

trainer = Trainer(
    model=model,
    args=args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    data_collator=data_collator,
    compute_metrics=compute_metrics
)

print("Metric for best model:", trainer.args.metric_for_best_model)
trainer.evaluate()


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

...

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Metric for best model: f1


compute_metrics called!
Result from metric.compute: {'f1': 0.6116487770543727}


{'eval_loss': 0.7161104083061218,
 'eval_model_preparation_time': 0.0014,
 'eval_f1': 0.6116487770543727,
 'eval_runtime': 9.245,
 'eval_samples_per_second': 426.716,
 'eval_steps_per_second': 53.434}

In [5]:

# ==========================
# 10. Train
# ==========================
trainer.train()

# ==========================
# 11. Final evaluation
# ==========================
trainer.evaluate()


Epoch,Training Loss,Validation Loss,Model Preparation Time,F1
1,0.214700,0.246562,0.001400,0.896951
2,0.141600,0.433069,0.001400,0.882720
3,0.058700,0.506387,0.001400,0.897603


compute_metrics called!
Result from metric.compute: {'f1': 0.8969512195121951}
compute_metrics called!
Result from metric.compute: {'f1': 0.8827199001871491}
compute_metrics called!
Result from metric.compute: {'f1': 0.8976034858387799}


compute_metrics called!
Result from metric.compute: {'f1': 0.8976034858387799}


{'eval_loss': 0.5063873529434204,
 'eval_model_preparation_time': 0.0014,
 'eval_f1': 0.8976034858387799,
 'eval_runtime': 8.6108,
 'eval_samples_per_second': 458.145,
 'eval_steps_per_second': 57.37,
 'epoch': 3.0}

In [4]:
# Commented out to avoid memory error
#os.environ["TOKENIZERS_PARALLELISM"] = "true"
#torch.set_float32_matmul_precision("high")